# Posture Scoring Model Training

This notebook trains an LSTM regression model to predict posture quality scores (0-100) from skeletal keypoint sequences.


In [ ]:
import os
import sys
import numpy as np

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import TensorFlow
import tensorflow as tf

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 1. Load Dataset


In [ ]:
from model.posture_dataset import PostureDataset
from model.feature_extractor import NUM_FEATURES, get_feature_names

# Configuration
DATA_DIR = "data/neck_stretch"
SEQUENCE_LENGTH = 30  # 30 frames per sequence (~1 second at 30fps)
BATCH_SIZE = 16
OVERLAP = 0.5  # 50% overlap between windows

# Load dataset
dataset = PostureDataset(
    data_dir=DATA_DIR,
    sequence_length=SEQUENCE_LENGTH,
    batch_size=BATCH_SIZE,
    shuffle=True,
    overlap=OVERLAP,
)

# Get all data for train/test split
X, y = dataset.get_all_data()
print(f"\nDataset shape: X={X.shape}, y={y.shape}")
print(f"Number of features: {NUM_FEATURES}")
print(f"Feature names: {get_feature_names()[:5]}...")

## 2. Train/Validation Split


In [ ]:
from sklearn.model_selection import train_test_split

# 80/20 split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"\nScore range: {y_train.min() * 100:.1f} - {y_train.max() * 100:.1f}")

## 3. Create Model


In [ ]:
from model.scoring_model import (
    create_scoring_model,
    compile_model,
    create_callbacks,
    get_model_summary,
)

# Create model
model = create_scoring_model(
    sequence_length=SEQUENCE_LENGTH,
    num_features=NUM_FEATURES,
    lstm_units=(64, 32),
    dropout_rate=0.3,
    dense_units=16,
    use_gru=False,  # Use LSTM
)

# Compile model
model = compile_model(
    model,
    learning_rate=0.001,
    loss="mse",
)

# Print summary
print(get_model_summary(model))

## 4. Train Model


In [ ]:
# Training configuration
EPOCHS = 100
SAVE_DIR = "save"

# Create callbacks
callbacks = create_callbacks(
    save_dir=SAVE_DIR,
    patience=15,
    min_delta=0.001,
)

# Train
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1,
)

## 5. Evaluate Model


In [ ]:
import matplotlib.pyplot as plt

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss plot
axes[0].plot(history.history["loss"], label="Train Loss")
axes[0].plot(history.history["val_loss"], label="Val Loss")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("MSE Loss")
axes[0].set_title("Training & Validation Loss")
axes[0].legend()
axes[0].grid(True)

# MAE plot
axes[1].plot(history.history["mae"], label="Train MAE")
axes[1].plot(history.history["val_mae"], label="Val MAE")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("MAE")
axes[1].set_title("Training & Validation MAE")
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "training_history.png"), dpi=150)
plt.show()

In [ ]:
# Final evaluation
val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)

print("\n" + "=" * 50)
print("FINAL MODEL EVALUATION")
print("=" * 50)
print(f"Validation MSE Loss: {val_loss:.4f}")
print(f"Validation MAE: {val_mae:.4f}")
print(f"Validation MAE (0-100 scale): {val_mae * 100:.2f}")

## 6. Save Final Model


In [ ]:
# Save final model
final_model_path = os.path.join(SAVE_DIR, "posture_scorer.h5")
model.save(final_model_path)
print(f"Model saved to: {final_model_path}")

# Also save as SavedModel format for better compatibility
savedmodel_path = os.path.join(SAVE_DIR, "posture_scorer_savedmodel")
model.save(savedmodel_path)
print(f"SavedModel saved to: {savedmodel_path}")

## 7. Test Inference


In [ ]:
# Load model for inference test
loaded_model = tf.keras.models.load_model(final_model_path)

# Make predictions on validation set
predictions = loaded_model.predict(X_val[:5], verbose=0)

print("\nSample Predictions vs Ground Truth:")
print("-" * 40)
for i, (pred, true) in enumerate(zip(predictions, y_val[:5])):
    print(f"Sample {i + 1}: Predicted={pred[0] * 100:.1f}, True={true * 100:.1f}")